# Multiple models comparison

In [ ]:
import boto3
from time import sleep
import pandas as pd
import seaborn as sns
import pprint
pp = pprint.PrettyPrinter(indent=2)

In [ ]:
forecast = boto3.client(service_name='forecast')
forecastquery = boto3.client(service_name='forecastquery')

In [ ]:
project = 'util_power_forecastdemo' # This should be the same as your previous notebook

In [ ]:
datasetGroupArn = 'arn:aws:forecast:eu-west-1:553700203877:dataset-group/util_power_forecastdemo_dsg'  # Fill in the quotes from the output of the previous notebook.

In [ ]:
algos = ['Prophet', 'ETS', 'Deep_AR_Plus']

predictors = {a:{} for a in algos}

for p in predictors:
    predictors[p]['predictor_name'] = project + '_' + p + '_algo'
    predictors[p]['algorithm_arn'] = 'arn:aws:forecast:::algorithm/' + p

pp.pprint(predictors)

In [ ]:
forecastHorizon = 24

### Create multiple predictors

In [ ]:
def create_predictor_response(pred_name, algo_arn, forecast_horizon):
    response=forecast.create_predictor(PredictorName=pred_name, 
                                       AlgorithmArn=algo_arn,
                                       ForecastHorizon=forecast_horizon,
                                       PerformAutoML= False,
                                       PerformHPO=False,
                                       EvaluationParameters= {"NumberOfBacktestWindows": 1, 
                                                              "BackTestWindowOffset": 24}, 
                                       InputDataConfig= {"DatasetGroupArn": datasetGroupArn},
                                       FeaturizationConfig= {"ForecastFrequency": "H", 
                                                             "Featurizations": 
                                                             [
                                                                 {"AttributeName": "target_value", 
                                                                  "FeaturizationPipeline": 
                                                                  [
                                                                      {"FeaturizationMethodName": "filling", 
                                                                       "FeaturizationMethodParameters": 
                                                                       {"frontfill": "none", 
                                                                        "middlefill": "zero", 
                                                                        "backfill": "zero"}
                                                                      }
                                                                  ]
                                                                 }
                                                             ]
                                                            }
                                      )
    return response

In [ ]:
for p in predictors.keys():
    predictor_response = create_predictor_response(predictors[p]['predictor_name'], predictors[p]['algorithm_arn'], forecastHorizon)
    
    predictorArn=predictor_response['PredictorArn']
    
    # wait for the predictor to be actually created
    print('------------------ Creating ' + p)
    while True:
        predictorStatus = forecast.describe_predictor(PredictorArn=predictorArn)['Status']
        print(predictorStatus)
        if predictorStatus != 'ACTIVE' and predictorStatus != 'CREATE_FAILED':
            sleep(30)
        else:
            predictors[p]['predictor_arn'] = predictorArn  # save it, just for reference
            break
            
    # compute accuracy metrics, then proceed with the next algorithm        
    predictors[p]['accuracy'] = forecast.get_accuracy_metrics(PredictorArn=predictorArn)

**TODO:** (Bar?)plot RMSE, 0.9-, 0.5- and 0.1-quantile LossValues for each algorithm

In [ ]:
pp.pprint(predictors)

In [ ]:
scores = pd.DataFrame(columns=['predictor', 'RMSE'])
for p in predictors:
    score = predictors[p]['accuracy']['PredictorEvaluationResults'][0]['TestWindows'][0]['Metrics']['RMSE']
    scores = scores.append(pd.DataFrame({'predictor':[p], 'RMSE':[score]}), ignore_index=True)

In [ ]:
fig = sns.barplot(data=scores, x='predictor', y='RMSE').set_title('Root Mean Square Error')